<a href="https://colab.research.google.com/github/Somanathan-R/ParallelProgrammingCUDA/blob/main/gpu_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
%%writefile update_directive_example.c

#include <stdio.h>
#include <stdlib.h>

#define N 10

int main() {
    float *a;
    int i;

    // Allocate memory for array 'a' on the host (CPU)
    a = (float *)malloc(N * sizeof(float));

    // Initialize array 'a' on the device (GPU)
    #pragma acc parallel loop
    for (i = 0; i < N; i++) {
        a[i] = i;
    }

    // Update array 'a' on the host with values from the device
    #pragma acc update self(a)

    // Print the updated array
    printf("Array a after update:\n");
    for (i = 0; i < N; i++) {
        printf("%f\n", a[i]);
    }

    // Free allocated memory
    free(a);

    return 0;
}


Overwriting update_directive_example.c


In [5]:
!gcc -o update_directive_example -fopenacc update_directive_example.c
!./update_directive_example


Array a after update:
0.000000
1.000000
2.000000
3.000000
4.000000
5.000000
6.000000
7.000000
8.000000
9.000000


In [11]:
%%writefile loop_optimization_example.c

#include <stdio.h>
#include <stdlib.h>

#define N 10

int main() {
    float A[N][N], B[N][N], C[N][N];
    int i, j;

    // Initialize matrices A and B
    printf("Matrix A:\n");
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++) {
            A[i][j] = i + j;
            printf("%6.1f ", A[i][j]);
        }
        printf("\n");
    }
    printf("\n");

    printf("Matrix B:\n");
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++) {
            B[i][j] = i - j;
            printf("%6.1f ", B[i][j]);
        }
        printf("\n");
    }
    printf("\n");

    // Compute element-wise addition of matrices A and B, storing the result in matrix C
    #pragma acc parallel loop collapse(2)
    for (i = 0; i < N; i++) {
        for (j = 0; j < N; j++) {
            C[i][j] = A[i][j] + B[i][j];
        }
    }

    // Print a few elements of matrix C to verify the computation
    printf("First few elements of matrix C:\n");
    for (i = 0; i < 5 && i < N; i++) {
        for (j = 0; j < 5 && j < N; j++) {
            printf("%6.1f ", C[i][j]);
        }
        printf("\n");
    }

    return 0;
}



Overwriting loop_optimization_example.c


In [12]:
!gcc -o loop_optimization_example -fopenacc loop_optimization_example.c
!./loop_optimization_example


Matrix A:
   0.0    1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0    9.0 
   1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0    9.0   10.0 
   2.0    3.0    4.0    5.0    6.0    7.0    8.0    9.0   10.0   11.0 
   3.0    4.0    5.0    6.0    7.0    8.0    9.0   10.0   11.0   12.0 
   4.0    5.0    6.0    7.0    8.0    9.0   10.0   11.0   12.0   13.0 
   5.0    6.0    7.0    8.0    9.0   10.0   11.0   12.0   13.0   14.0 
   6.0    7.0    8.0    9.0   10.0   11.0   12.0   13.0   14.0   15.0 
   7.0    8.0    9.0   10.0   11.0   12.0   13.0   14.0   15.0   16.0 
   8.0    9.0   10.0   11.0   12.0   13.0   14.0   15.0   16.0   17.0 
   9.0   10.0   11.0   12.0   13.0   14.0   15.0   16.0   17.0   18.0 

Matrix B:
   0.0   -1.0   -2.0   -3.0   -4.0   -5.0   -6.0   -7.0   -8.0   -9.0 
   1.0    0.0   -1.0   -2.0   -3.0   -4.0   -5.0   -6.0   -7.0   -8.0 
   2.0    1.0    0.0   -1.0   -2.0   -3.0   -4.0   -5.0   -6.0   -7.0 
   3.0    2.0    1.0    0.0   -1.0   -2.0   -3.0   -4.0 